## 1. Импорт и установка библиотек

In [ ]:
# Установка библиотек
pip install pyTelegramBotAPI

In [90]:
# Импорт всех необходимых библиотек
import telebot
from telebot import types
import requests
import sqlite3

## 2. Глобальные переменные

In [91]:
API_KEY = 1 # API-КЛЮЧ chatGPT
API_BOT = '6014215162:AAE3WpfnXjnmpKgWTJn5JEi13WtlvaTiV10' # API-КЛЮЧ бота телеграм

## 3. Дополнительные функции

In [92]:
# Функция для отправки запросов chatGPT
def ask_gpt3(text):

    headers = {
        'Authorization': f'Bearer {API_KEY}',
        'Content-Type': 'application/json',
    }
    data = {
        "prompt": f"{text} обработай это предложение следующим форматом: 1) Выведи оригинал предложение 2) Выведи перевод предложения на русском языке 3) Выдели слова-реалии из предложения на китайском языке 4) Дай каждой реалии справку и перевод. ",
        "max_tokens": 500
    }
    response = requests.post('https://api.openai.com/v1/engines/davinci-codex/completions', json=data, headers=headers)
    if response.status_code == 200:
        return response.json()['choices'][0]['text'].strip()
    else:
        return "Произошла ошибка во время обработки запроса."

## 4. Основной код бота

In [93]:
# Инициируем бота
bot = telebot.TeleBot(API_BOT)

In [94]:
# Создание начальной команды /start. При введении в чат пользователем команды '/start'
# инициируется запуск функции def main(message)
@bot.message_handler(commands=['start'])
def main(message):
# Создание "шаблона" (контейнера) в который мы будем вставлять кнопки
    markup = types.InlineKeyboardMarkup()
# Создание первой кнопки, параметр text - текст кнопки, параметр callback_data - "триггер", создаваемый кнопкой,
# который перехватывается специальными функциями под декоратором (@bot.callback_query_handler) 
    btn1 = types.InlineKeyboardButton(
        text='1. Выучить новое слово-реаль',
        callback_data='send'
    )
# Создание второй кнопки  
    btn2 = types.InlineKeyboardButton(
        text='2. Найти слово-реаль', 
        callback_data='tran'
    )
# Вставка кнопок в контейнер
    markup.add(btn1, btn2)
# Отправка сообщения пользователю. Первый параметр (chat_id) - уникальный id чата, в который будут
# отправляться сообщения, второй параметр (text) - текст сообщения, третий параметр (reply_markup) - дополнительные 
# элементы интерфейса сообщения.
    bot.send_message(
        message.chat.id,
        'Привет, пользователь! Я — ваш помощник в изучении китайского языка. Я могу помочь вам найти и перевести слова-реалии в предложении, а также выдать случайное слово-реалию. Что вы хотите сделать?', 
        reply_markup=markup
        )

In [95]:
# Реализация кнопки '1. Выучить новое слово-реаль'.
# bot.callback_query_handler(func=lambda call: call.data == 'send') перехватывает сингнал (триггер) 'send', создаваемый кнопкой '1. Выучить новое слово-реаль'
# и инициирует запуск функции def send_table(call)
@bot.callback_query_handler(func=lambda call: call.data == 'send')
def send_table(call):
# Удаление предыдущего сообщения    
    bot.delete_message(call.message.chat.id, call.message.message_id)
# Создание контейнера для кнопок 
    markup1 = types.InlineKeyboardMarkup()
# Создание первой кнопки
    btn_1 = types.InlineKeyboardButton(
        '1. На главную', 
        callback_data = 'start'
    )
# Создание второй кнопки
    btn_2 = types.InlineKeyboardButton(
        '2. Ещё одно слово', 
        callback_data = 'send'
    )
# Вставка кнопок в контейнер
    markup1.add(btn_1, btn_2)
# Подключение к локальной базе данных
    conn = sqlite3.connect('base_tran3.sql')
# Подключение курсора управления базой данных
    cur = conn.cursor()
# Выполнение SQL-запроса. SQL-запрос выводит случайную реаль из локальной базы данных
    cur.execute('SELECT description FROM trans ORDER BY RANDOM() LIMIT 1')
    word = cur.fetchall()
# Отключение от базы данных и закрытие курсора управления
    cur.close()
    conn.close()
# Отправка сообщения пользователю со словом-реалью из локальной базы данных
    bot.send_message(
        call.message.chat.id, 
        word, 
        reply_markup=markup1
    )   

In [96]:
# Реализация кнопки '2. Найти слово-реаль'
@bot.callback_query_handler(func=lambda call: call.data == 'tran')
def prompt(call):
# Отправка сообщения
    bot.delete_message(call.message.chat.id, call.message.message_id)
    message = bot.send_message(call.message.chat.id, 'Введите предложение со словами-реалями:')
# Метод .register_next_step_handler регестрирует сообщение (указывается первым параметром), которое отправил бот или пользователь
# и начинает выполнять функцию, которая указана вторым параметром
    bot.register_next_step_handler(message, handle_user_message)

def handle_user_message(message):
# Отправка запроса и получение ответа от ChatGPT 
    response = ask_gpt3(message.text)

    markup1 = types.InlineKeyboardMarkup()
    
    btn_1 = types.InlineKeyboardButton(
        '1. На главную', 
        callback_data = 'start'
    )
    
    btn_2 = types.InlineKeyboardButton(
        '2. Ещё одно предложение', 
        callback_data = 'tran'
    )
    
    markup1.add(btn_1, btn_2)
    
    bot.send_message(
        message.chat.id, 
        response, 
        reply_markup=markup1
    )

In [97]:
# Создание функции возвращения на главное меню 
@bot.callback_query_handler(func=lambda call: call.data == 'start')
def restart(call):
    bot.delete_message(call.message.chat.id, call.message.message_id)
    main(call.message)

### Создание команд разработчика 

In [98]:
# Функция для создания локальной базы данных через бота telegram
@bot.message_handler(commands=['create_table'])
def create_table(message):
    conn = sqlite3.connect('base_tran3.sql')
    cur = conn.cursor()
# Создание локальной базы данных под именем trans, только в том случае, если её ещё нет 
    cur.execute('CREATE TABLE IF NOT EXISTS trans (id int auto_increment primary key, name varchar(25), description varchar(500))')
    conn.commit()
    
    cur.close()
    conn.close()

    bot.send_message(message.chat.id, 'Таблица создана!')

In [99]:
# Функции для добавления слов в базу данных
@bot.message_handler(commands=['add'])
def add_table(message):
    bot.send_message(message.chat.id, 'Введите слово-реаль на китайском языке')
    bot.register_next_step_handler(message, add_name)

def add_name(message):

    name = message.text
    
    bot.send_message(message.chat.id, 'Напишите описание')
    
    bot.register_next_step_handler(
        message, 
        add_description,
        name
    )

def add_description(message, name):
    
    desp = message.text
    
    conn = sqlite3.connect('base_tran3.sql')
    cur = conn.cursor()
    cur.execute('INSERT INTO trans (name, description) VALUES ("%s", "%s")' %(name, desp))
    conn.commit()
    cur.close()
    conn.close()
    
    markup1 = types.InlineKeyboardMarkup()
    
    btn_1 = types.InlineKeyboardButton(
        '1. Дабавить ещё', 
        callback_data = 'add_more'
    )
    
    markup1.add(btn_1)
    
    bot.send_message(
        message.chat.id, 
        'Слово-реаль добавлена в базу данных', 
        reply_markup=markup1
    )


@bot.callback_query_handler(func=lambda call: call.data == 'add_more')
def callback_add_more(call):
    bot.delete_message(call.message.chat.id, call.message.message_id)
    bot.answer_callback_query(call.id)
    add_table(call.message)

In [ ]:
# Метод для поллинга бота
bot.polling(non_stop=True)